In [1]:
#runs FFT and loads mel spec images to be used as inputs for CNN in modeling step

In [2]:
import numpy as np
import pandas as pd
import os
import gc
import cv2
from joblib import Parallel, delayed
from tqdm.notebook import tqdm

from pydub import AudioSegment as AS
import librosa
from librosa.feature import melspectrogram
from librosa.core import power_to_db as ptdb

from scipy.ndimage.morphology import binary_dilation, binary_erosion
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences as pad

In [3]:
os.listdir('birdsong-recognition')

['example_test_audio',
 'example_test_audio_metadata.csv',
 'example_test_audio_summary.csv',
 'mel spec images',
 'sample_submission.csv',
 'test.csv',
 'train.csv',
 'train_audio']

In [4]:
os.getcwd()

'C:\\Users\\jonat\\Desktop\\Career\\Springboard\\Capstone 3'

In [5]:
test_path = r'C:\Users\jonat\Desktop\Career\Springboard\Capstone 3\birdsong-recognition\test.csv'
train_path = r'C:\Users\jonat\Desktop\Career\Springboard\Capstone 3\birdsong-recognition\train.csv'
test_audio_path = r'C:\Users\jonat\Desktop\Career\Springboard\Capstone 3\birdsong-recognition\example_test_audio/'
train_audio_path = r'C:\Users\jonat\Desktop\Career\Springboard\Capstone 3\birdsong-recognition\train_audio/'

In [6]:
test_d = pd.read_csv(test_path)
train_d = pd.read_csv(train_path)

In [7]:
train_d['file_path'] = train_audio_path + '/' + train_d['ebird_code'] + '/' + train_d['filename']

In [8]:
train_d['file_path'][0]

'C:\\Users\\jonat\\Desktop\\Career\\Springboard\\Capstone 3\\birdsong-recognition\\train_audio//aldfly/XC134874.mp3'

In [9]:
test = r'C:\Users\jonat\Desktop\Career\Springboard\Capstone 3\birdsong-recognition\train_audio\aldfly'

In [10]:
keys = set(train_d.ebird_code)
values = np.arange(0,len(keys))
dict_code = dict(zip(sorted(keys),values))

In [11]:
print(dict_code)

{'aldfly': 0, 'ameavo': 1, 'amebit': 2, 'amecro': 3, 'amegfi': 4, 'amekes': 5, 'amepip': 6, 'amered': 7, 'amerob': 8, 'amewig': 9, 'amewoo': 10, 'amtspa': 11, 'annhum': 12, 'astfly': 13, 'baisan': 14, 'baleag': 15, 'balori': 16, 'banswa': 17, 'barswa': 18, 'bawwar': 19, 'belkin1': 20, 'belspa2': 21, 'bewwre': 22, 'bkbcuc': 23, 'bkbmag1': 24, 'bkbwar': 25, 'bkcchi': 26, 'bkchum': 27, 'bkhgro': 28, 'bkpwar': 29, 'bktspa': 30, 'blkpho': 31, 'blugrb1': 32, 'blujay': 33, 'bnhcow': 34, 'boboli': 35, 'bongul': 36, 'brdowl': 37, 'brebla': 38, 'brespa': 39, 'brncre': 40, 'brnthr': 41, 'brthum': 42, 'brwhaw': 43, 'btbwar': 44, 'btnwar': 45, 'btywar': 46, 'buffle': 47, 'buggna': 48, 'buhvir': 49, 'bulori': 50, 'bushti': 51, 'buwtea': 52, 'buwwar': 53, 'cacwre': 54, 'calgul': 55, 'calqua': 56, 'camwar': 57, 'cangoo': 58, 'canwar': 59, 'canwre': 60, 'carwre': 61, 'casfin': 62, 'caster1': 63, 'casvir': 64, 'cedwax': 65, 'chispa': 66, 'chiswi': 67, 'chswar': 68, 'chukar': 69, 'clanut': 70, 'cliswa': 

In [12]:
N = 8 # cores to split, for parallel processing later
CHUNKS = 1
POP_FRAC = 0.25
MAXLEN = 1000000
AMPLITUDE = 1000
CHUNK_SIZE = 500000
sr = 44100
n_mels = 128
test_sample_rate = 320000

In [12]:
train_d.ebird_code[0]

'aldfly'

In [13]:
def normalize(x):
    return np.float32(x)/(2**15)

def read(data,norm=False):
    try:
        a = AS.from_mp3(data).set_frame_rate(test_sample_rate)
    except:
        return test_sample_rate, np.zeros(MAXLEN)
    
    y = np.array(a.get_array_of_samples())
    
    if norm: 
        return a.frame_rate, normalize(y)
    else:
        return a.frame_rate, np.float32(y) 
    
def write(file,sr,x,norm=False):
    birds_audio_rate, format = '320k', 'mp3'
    ch = 2 if (x.ndim==2 and x.shape[1] == 2) else 1
    y = np.int16(x*2**15) if normalized else np.int16(x)
    song = AS(y.tobytes(),frame_rate = sr, sample_width = 2, channels = ch)
    song.export(file, format=file_format, bitrate = birds_audio_rate)

In [14]:
#testing copy
def read(file, norm=False):
    try:
        a = AS.from_mp3(file)
        a = a.set_frame_rate(test_sample_rate)
    except:
        return test_sample_rate, np.zeros(MAXLEN)

    y = np.array(a.get_array_of_samples())
    if a.channels == 2: y = y.reshape((-1, 2))
    if norm: return a.frame_rate, normalize(y)
    if not norm: return a.frame_rate, np.float32(y)

In [17]:
def get_len(length):
    if length > MAXLEN: return MAXLEN
    if length <= MAXLEN: return int(length*POP_FRAC)
    
def get_index(length):
    length = get_len(length)
    max_index = MAXLEN - CHUNK_SIZE
    ids = np.random.randint(length + 1)
    chunk_range = ids,ids + CHUNK_SIZE
    chunk_idx = min([chunk_range[1],max_index])
    return (chunk_idx,chunk_idx + CHUNK_SIZE)
    #return max_index, max_index + CHUNK_SIZE

In [18]:
def get_chunk(data,length):
    index = get_index(length)
    return data[index[0]:index[1]]

def audio_signal(data):
    length = max(data.shape)
    data = data.T.flatten().reshape(1,-1)
    data = np.float32(pad(data, maxlen=MAXLEN).reshape(-1))
    return [get_chunk(data,length) for _ in range(CHUNKS)]

In [19]:
def dilation(x,e):
    dilated = binary_dilation(e.sum(axis=0)>0, iterations = 3)
    x = x[np.round(np.interp(np.arange(x.shape[0]),
                             np.arange(dilated.shape[0])*x.shape[0]/dilated.shape[0],
                             dilated)).astype(bool)]
    return x

def erosion(m):
    norm_m=(m-m.min())/(m.max()-m.min())
    column_medians=np.median(norm_m,axis=0)
    row_medians=np.median(norm_m)
    eroded_spectrogram = binary_erosion(np.greater(norm_m,column_medians*3)&np.greater(norm_m.T,row_medians*3).T*1)
    return eroded_spectrogram

In [20]:
def to_imagenet(X, mean = None, std = None, norm_max = None, norm_min = None, epsilon = 1e-6):
    mean = mean or X.mean()
    std = std or X.std()
    Xstd = (X - mean )/ (std + epsilon)
    _min, _max = Xstd.min(), Xstd.max()
    norm_max = norm_max or _max
    norm_min = norm_min or _min
    if (_max - _min) > epsilon:
        V = Xstd
        V[V<norm_min] = norm_min
        V[V>norm_max] = norm_max
        V = 255*((V - norm_min) / (norm_max - norm_min))
    else:
        V = np.zeros_like(Xstd, dtype = np.uint8)
    return np.stack([V]*3, axis = -1) #gray image in all three RGB channels

In [21]:
def get_mel(data):
    melsp = melspectrogram(data,n_mels=n_mels)
    #dilated = dilation(data,melsp)
    #return to_imagenet(erosion(ptdb(melsp)))
    return to_imagenet(ptdb(melsp))

def get_melsp_im(data):
    data = audio_signal(data)
    return np.stack([get_mel(d)] for d in data)

In [22]:
def save_image(indices, path):
    folder = train_audio_path
    
    for i in tqdm(indices):
        files = train_d.filename[i]
        ebird_code = train_d.ebird_code[i]
        
        default_signal = np.random.random(MAXLEN)*AMPLITUDE
        default_value = sr, np.int32(np.round(default_signal))
        
        values = read(folder + ebird_code + '/' + files)
        _ , data = values if len(values) == 2 else default_value
        
        image = np.nan_to_num(get_melsp_im(data))[0]
        print(image.shape)
        if not cv2.imwrite(files+'.jpg', image[0,:,:,:]):
            raise Exception('Could not write image');
        else:
            cv2.imwrite(files+'.jpg', image[0,:,:,:]);
            del image;
            gc.collect();

In [13]:
train_ids = np.array_split(np.arange(len(train_d)), 10)
train_ids_1, train_ids_2, train_ids_3, train_ids_4, train_ids_5,train_ids_6, train_ids_7, train_ids_8, train_ids_9, train_ids_10 = train_ids

train_ids_1 = np.array_split(np.array(train_ids_1), N)
train_ids_2 = np.array_split(np.array(train_ids_2), N)

In [26]:
train_ids_2[7]

array([4009, 4010, 4011, 4012, 4013, 4014, 4015, 4016, 4017, 4018, 4019,
       4020, 4021, 4022, 4023, 4024, 4025, 4026, 4027, 4028, 4029, 4030,
       4031, 4032, 4033, 4034, 4035, 4036, 4037, 4038, 4039, 4040, 4041,
       4042, 4043, 4044, 4045, 4046, 4047, 4048, 4049, 4050, 4051, 4052,
       4053, 4054, 4055, 4056, 4057, 4058, 4059, 4060, 4061, 4062, 4063,
       4064, 4065, 4066, 4067, 4068, 4069, 4070, 4071, 4072, 4073, 4074,
       4075, 4076, 4077, 4078, 4079, 4080, 4081, 4082, 4083, 4084, 4085,
       4086, 4087, 4088, 4089, 4090, 4091, 4092, 4093, 4094, 4095, 4096,
       4097, 4098, 4099, 4100, 4101, 4102, 4103, 4104, 4105, 4106, 4107,
       4108, 4109, 4110, 4111, 4112, 4113, 4114, 4115, 4116, 4117, 4118,
       4119, 4120, 4121, 4122, 4123, 4124, 4125, 4126, 4127, 4128, 4129,
       4130, 4131, 4132, 4133, 4134, 4135, 4136, 4137, 4138, 4139, 4140,
       4141, 4142, 4143, 4144, 4145, 4146, 4147, 4148, 4149, 4150, 4151,
       4152, 4153, 4154, 4155, 4156, 4157, 4158, 41

In [27]:
os.chdir(r'C:\Users\jonat\Desktop\Career\Springboard\Capstone 3\birdsong-recognition\mel spec images')

In [29]:
!mkdir train_file_2

In [25]:
path = 'train_file_1/'
os.chdir(os.path.join(r'C:\Users\jonat\Desktop\Career\Springboard\Capstone 3\birdsong-recognition\mel spec images',path))
parallel = Parallel(n_jobs=8,backend='threading')
parallel(delayed(save_image)(id,path) for id in train_ids_1)

C:\Users\jonat\anaconda3\envs\torchEnv\lib\site-packages\joblib\parallel.py:263: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  for func, args, kwargs in self.items]


(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)(1, 128, 977, 3)

(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 

(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 

(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)(1, 128, 977, 3)

(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)(1, 128, 977, 3)

(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 

(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)(1, 128, 977, 3)

(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 

(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)(1, 128, 977, 3)

(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 

[None, None, None, None, None, None, None, None]

In [26]:
path = 'train_file_2/'
os.chdir(os.path.join(r'C:\Users\jonat\Desktop\Career\Springboard\Capstone 3\birdsong-recognition\mel spec images',path))
parallel = Parallel(n_jobs=8,backend='threading')
parallel(delayed(save_image)(id,path) for id in train_ids_2)

(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)(1, 128, 977, 3)

(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)(1, 128, 977, 3

(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)(1, 128, 977, 3)

(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 

(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 

(1, 128, 977, 3)
(1, 128, 977, 3)(1, 128, 977, 3)

(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 

(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 3)
(1, 128, 977, 

[None, None, None, None, None, None, None, None]